In [1]:
import torch
import numpy as np
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

# preparetions of dataset
import os
from os import listdir
from os.path import isfile, join
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision
from skimage import io



random.seed(42)
tf.random.set_seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Creating dataset

In [2]:
class CreateDataset(Dataset):
    def __init__(self, csv_file, root_dir, trasform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.trasform = trasform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)

        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

        if self.trasform:
            image = self.trasform(image)
            image = torchvision.transforms.functional.rgb_to_grayscale(image, num_output_channels=3)
            

        return (image, y_label)

In [3]:
onlyfiles = ['./images/Blur_Positive_target//'+f for f in listdir('./images/Blur_Positive_target/') if isfile(join('./images/Blur_Positive_target', f))]

dataset = pd.DataFrame(onlyfiles, columns=['path'])
dataset['target'] = 1
onlyfiles = ['./images/Blur_Empty_table/'+f for f in listdir('./images/Blur_Empty_table/') if isfile(join('./images/Blur_Empty_table/', f))]
second = pd.DataFrame(onlyfiles, columns=['path'])
second['target'] = 0
print(dataset.shape, second.shape)
finaldataset = pd.concat([dataset, second])

finaldataset.shape

(8921, 2) (28762, 2)


(37683, 2)

In [4]:
finaldataset.to_csv("Gray_Blured_Data.csv", index=False)
dataset = CreateDataset(csv_file='./Gray_Blured_Data.csv', root_dir='./',
                       trasform=transforms.ToTensor())
train, valid, test = torch.utils.data.random_split(dataset, [24494, 3768, 9421])
train_loader = DataLoader(dataset=train, batch_size=24494, shuffle=True)
valid_loader = DataLoader(dataset=train, batch_size=3768, shuffle=True)
test_loader = DataLoader(dataset=test, batch_size=9421, shuffle=True)


In [5]:
for batch, (data, targets) in enumerate(train_loader):
    print(data.shape)
    X_train = data
    y_train = targets
print(f'Train:  x={X_train.shape} y={y_train.shape}')

for batch, (data, targets) in enumerate(valid_loader):
    X_valid = data
    y_valid = targets
print(f'Validation:  x={X_valid.shape} y={y_valid.shape}')
    
for batch, (data, targets) in enumerate(test_loader):
    X_test = data
    y_test = targets
print(f'Test:  x={X_test.shape} y={y_test.shape}')

torch.Size([24494, 3, 116, 116])
Train:  x=torch.Size([24494, 3, 116, 116]) y=torch.Size([24494])
Validation:  x=torch.Size([1886, 3, 116, 116]) y=torch.Size([1886])
Test:  x=torch.Size([9421, 3, 116, 116]) y=torch.Size([9421])


### Data to grayscale (shape = (1, 116, 116))

In [6]:
transform_to_gray = torchvision.transforms.Grayscale(num_output_channels=1)
X_train = transform_to_gray(X_train)
print('Train: ', X_train.shape)

transform_to_gray = torchvision.transforms.Grayscale(num_output_channels=1)
X_valid = transform_to_gray(X_valid)
print('Validation: ', X_valid.shape)

transform_to_gray = torchvision.transforms.Grayscale(num_output_channels=1)
X_test = transform_to_gray(X_test)
print('Test: ', X_test.shape)

Train:  torch.Size([24494, 1, 116, 116])
Validation:  torch.Size([1886, 1, 116, 116])
Test:  torch.Size([9421, 1, 116, 116])


In [7]:
# numpy to tensor
y_train = y_train.to(torch.float)
y_valid = y_valid.to(torch.float)
X_train = torch.FloatTensor(X_train)
X_valid = torch.FloatTensor(X_valid)

# Neural_Network

In [8]:
# Simplest model on Pytorch (there is no need in convolution)
# MaxPool2d -> Linear -> Sigmoid
class Revision(torch.nn.Module):
    def __init__(self):
        super(Revision, self).__init__()
        
        self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = torch.nn.Linear(58*58, 1)
        self.out = torch.nn.Sigmoid()
        
    def forward(self, x):
        x = self.pool2(x)
        # растягиваем вектор х
        x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))
        
        x = self.fc1(x)
        x = self.out(x)
        
        return x
    
final_v4 = Revision()

In [9]:
# optimizer and loss function
optimizer = torch.optim.Adam(final_v4.parameters(), amsgrad=True, lr=0.001)
loss = torch.nn.BCELoss()

In [10]:
# fit model
batch_size = 100
test_accuracy_history = []
test_loss_history = []

for epoch in range(65):
    # create batches
    order = np.random.permutation(len(X_train))

    for start_index in range(0, len(X_train), batch_size):
        # fit model
        optimizer.zero_grad()
        final_v4.train()
        batch_indexes = order[start_index:start_index+batch_size]

        X_batch = X_train[batch_indexes]
        y_batch = y_train[batch_indexes]

        preds = final_v4.forward(X_batch).squeeze()
        loss_value = loss(preds, y_batch)
        loss_value.backward()

        optimizer.step()
        
    # evaluate on valid data
    final_v4.eval()
    test_preds = final_v4.forward(X_valid).squeeze()
    test_loss_history.append(loss(test_preds,y_valid).data)

    accuracy = (torch.round(test_preds) == y_valid).float().mean().data
    test_accuracy_history.append(accuracy)
    if accuracy == 0.9995:
        break

    print(accuracy)

tensor(0.9894)
tensor(0.9915)
tensor(0.9920)
tensor(0.9920)
tensor(0.9920)
tensor(0.9958)
tensor(0.9920)
tensor(0.9958)
tensor(0.9963)
tensor(0.9947)
tensor(0.9968)
tensor(0.9968)
tensor(0.9968)
tensor(0.9968)
tensor(0.9968)
tensor(0.9947)
tensor(0.9968)
tensor(0.9968)
tensor(0.9968)
tensor(0.9968)
tensor(0.9968)
tensor(0.9968)
tensor(0.9973)
tensor(0.9973)
tensor(0.9973)
tensor(0.9973)
tensor(0.9968)
tensor(0.9973)
tensor(0.9973)
tensor(0.9968)
tensor(0.9968)
tensor(0.9979)
tensor(0.9973)
tensor(0.9979)
tensor(0.9973)
tensor(0.9979)
tensor(0.9973)
tensor(0.9973)
tensor(0.9979)
tensor(0.9973)
tensor(0.9973)
tensor(0.9979)
tensor(0.9973)
tensor(0.9979)
tensor(0.9979)
tensor(0.9979)
tensor(0.9979)
tensor(0.9973)
tensor(0.9973)
tensor(0.9979)
tensor(0.9979)
tensor(0.9973)
tensor(0.9973)
tensor(0.9979)
tensor(0.9979)
tensor(0.9979)
tensor(0.9984)
tensor(0.9979)
tensor(0.9973)
tensor(0.9984)
tensor(0.9979)
tensor(0.9973)
tensor(0.9973)
tensor(0.9979)
tensor(0.9973)


In [11]:
final_v4.eval()
test_preds = final_v4.forward(X_test).squeeze()
accuracy = (torch.round(test_preds) == y_test).float().mean().data
accuracy

tensor(0.9983)

# Same, but on keras

In [12]:
X_train_tf = tf.constant(X_train.numpy(), dtype=tf.float16)
X_test_tf = tf.constant(X_test.numpy(), dtype=tf.float16)
y_test_tf = tf.constant(y_test.numpy(), dtype=tf.float16)
y_train_tf = tf.constant(y_train.numpy(), dtype=tf.float16)
X_valid_tf = tf.constant(X_valid.numpy(), dtype=tf.float16)
y_valid_tf = tf.constant(y_valid.numpy(), dtype=tf.float16)

2022-12-18 19:05:06.596370: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-12-18 19:05:06.598309: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-18 19:05:06.600971: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [13]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(1, 116, 116)),
    tf.keras.layers.Permute((2,3,1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid')]
)

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute (Permute)            (None, 116, 116, 1)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 58, 58, 1)         0         
_________________________________________________________________
flatten (Flatten)            (None, 3364)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 3365      
Total params: 3,365
Trainable params: 3,365
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(optimizer='adam',
              loss= tf.losses.BinaryCrossentropy(),
              metrics=['accuracy']
)

In [16]:
history = model.fit(
    X_train_tf,
    y_train_tf,
    batch_size=100,
    epochs= 40,
    validation_data=(X_valid_tf, y_valid_tf)
)

Epoch 1/40


2022-12-18 19:05:08.821375: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-12-18 19:05:08.848157: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1190400000 Hz


245/245 [==============================] - 11s 42ms/step - loss: 0.1673 - accuracy: 0.9289 - val_loss: 0.0384 - val_accuracy: 0.9920
Epoch 2/40
245/245 [==============================] - 9s 39ms/step - loss: 0.0332 - accuracy: 0.9926 - val_loss: 0.0311 - val_accuracy: 0.9920
Epoch 3/40
245/245 [==============================] - 10s 41ms/step - loss: 0.0249 - accuracy: 0.9941 - val_loss: 0.0276 - val_accuracy: 0.9931
Epoch 4/40
245/245 [==============================] - 11s 43ms/step - loss: 0.0241 - accuracy: 0.9936 - val_loss: 0.0276 - val_accuracy: 0.9920
Epoch 5/40
245/245 [==============================] - 10s 41ms/step - loss: 0.0201 - accuracy: 0.9947 - val_loss: 0.0271 - val_accuracy: 0.9920
Epoch 6/40
245/245 [==============================] - 10s 42ms/step - loss: 0.0222 - accuracy: 0.9940 - val_loss: 0.0228 - val_accuracy: 0.9942
Epoch 7/40
245/245 [==============================] - 10s 42ms/step - loss: 0.0174 - accuracy: 0.9958 - val_loss: 0.0301 - val_accuracy: 0.9910
Epoc

In [17]:
final_v4.eval()
torch_preds = final_v4.forward(X_test).squeeze().detach().numpy()

print("Pytorch")
print(np.corrcoef([torch_preds, y_test.numpy()]))


tensorflow_preds = model.predict(X_test_tf).squeeze()
print("Tensorflow")
print(np.corrcoef([tensorflow_preds, y_test.numpy()]))

Pytorch
[[1.         0.99563161]
 [0.99563161 1.        ]]
Tensorflow
[[1.         0.99604904]
 [0.99604904 1.        ]]
